In [6]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from torch import optim
from torch.autograd import Variable

In [7]:
def load_mnist():
    train = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
    test = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
    return train, test

In [8]:
train, test = load_mnist()

In [9]:
#plot image in dataset

#plt.imshow(train.data[4], cmap='gray')
#plt.title('%i' % train.targets[4])
#plt.show()

In [10]:
loaders = {'train': DataLoader(train, batch_size=100, shuffle=True, num_workers=1), 'test': DataLoader(test, batch_size=100, shuffle=False, num_workers=1)}

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(1, 16, kernel_size=5, stride = 1, padding=2), nn.ReLU(), nn.MaxPool2d(2))
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=5, stride = 1, padding=2), nn.ReLU(), nn.MaxPool2d(2))

        self.out = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)

        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output, x

In [12]:
cnn = CNN()
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.01)
n_epochs = 10

In [13]:
def train(n_epochs, cnn, loaders):
    cnn.train()
    total_step = len(loaders['train'])
    for epoch in range(n_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            
            images = Variable(images)
            labels = Variable(labels)

            optimizer.zero_grad()
            outputs, _ = cnn(images)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()

            if (i + 1) % 100 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, n_epochs, i + 1, total_step, loss.item()))

In [14]:
train(n_epochs, cnn, loaders)

Epoch [1/10], Step [100/600], Loss: 0.1288
Epoch [1/10], Step [200/600], Loss: 0.0368
Epoch [1/10], Step [300/600], Loss: 0.1130
Epoch [1/10], Step [400/600], Loss: 0.1208
Epoch [1/10], Step [500/600], Loss: 0.0446
Epoch [1/10], Step [600/600], Loss: 0.0675
Epoch [2/10], Step [100/600], Loss: 0.0150
Epoch [2/10], Step [200/600], Loss: 0.1128
Epoch [2/10], Step [300/600], Loss: 0.0130
Epoch [2/10], Step [400/600], Loss: 0.0567
Epoch [2/10], Step [500/600], Loss: 0.0518
Epoch [2/10], Step [600/600], Loss: 0.0200
Epoch [3/10], Step [100/600], Loss: 0.0479
Epoch [3/10], Step [200/600], Loss: 0.0443
Epoch [3/10], Step [300/600], Loss: 0.0436
Epoch [3/10], Step [400/600], Loss: 0.0177
Epoch [3/10], Step [500/600], Loss: 0.0324
Epoch [3/10], Step [600/600], Loss: 0.0031
Epoch [4/10], Step [100/600], Loss: 0.0857
Epoch [4/10], Step [200/600], Loss: 0.0436
Epoch [4/10], Step [300/600], Loss: 0.0845
Epoch [4/10], Step [400/600], Loss: 0.0620
Epoch [4/10], Step [500/600], Loss: 0.0163
Epoch [4/10

In [15]:
def test():
    cnn.eval()
   
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in loaders['test']:
            images = Variable(images)
            outputs, _ = cnn(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()

        print('Test Accuracy: {} %'.format(100 * correct / total))

In [16]:
test()

Test Accuracy: 98.70999908447266 %
